# 基于命题分块以增强RAG

命题分块技术（Proposition Chunking）——这是一种通过将文档分解为原子级事实陈述来实现更精准检索的先进方法。与传统仅按字符数分割文本的分块方式不同，命题分块能保持单个事实的语义完整性。

-----
命题分块通过以下方式实现更精确的检索：

1. 将内容分解为原子化的、自包含的事实陈述
2. 创建更小、更细粒度的检索单元
3. 实现查询与相关内容间更精确的匹配
4. 过滤掉低质量或不完整的命题

-----
实现步骤：
- 从PDF文件中提取文本
- 从提取的文本创建分块
- 分块生成命题
- 为每个命题进行评估

构建一个不依赖LangChain或FAISS的完整实现方案。

In [1]:
import fitz
import os
import re
import json
import numpy as np
from tqdm import tqdm
from openai import OpenAI
from dotenv import load_dotenv
from datetime import datetime

load_dotenv()

True

In [2]:
client = OpenAI(
    base_url=os.getenv("LLM_BASE_URL"),
    api_key=os.getenv("LLM_API_KEY")
)
llm_model = os.getenv("LLM_MODEL_ID")
embedding_model = os.getenv("EMBEDDING_MODEL_ID")

pdf_path = "../../data/AI_Information.en.zh-CN.pdf"

In [3]:
def extract_text_from_pdf(pdf_path):
    """
    从 PDF 文件中提取文本，并打印前 `num_chars` 个字符。

    Args:
    pdf_path (str): Path to the PDF file.

    Returns:
    str: Extracted text from the PDF.
    """
    # 打开 PDF 文件
    mypdf = fitz.open(pdf_path)
    all_text = ""  # 初始化一个空字符串以存储提取的文本

    # Iterate through each page in the PDF
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]
        text = page.get_text("text")  # 从页面中提取文本
        all_text += text  # 将提取的文本追加到 all_text 字符串中

    return all_text  # 返回提取的文本

In [4]:
def chunk_text(text, chunk_size=800, overlap=100):
    """
    将文本分割为重叠的块。

    Args:
        text (str): 要分割的输入文本
        chunk_size (int): 每个块的字符数
        overlap (int): 块之间的字符重叠数

    Returns:
        List[Dict]: 包含文本和元数据的块字典列表
    """
    chunks = []  # 初始化一个空列表来存储块

    # 使用指定的块大小和重叠迭代文本
    for i in range(0, len(text), chunk_size - overlap):
        chunk = text[i:i + chunk_size]  # 提取指定大小的块
        if chunk:  # 确保不添加空块
            chunks.append({
                "text": chunk,  # 块文本
                "chunk_id": len(chunks) + 1,  # 块的唯一ID
                "start_char": i,  # 块的起始字符索引
                "end_char": i + len(chunk)  # 块的结束字符索引
            })

    print(f"创建了 {len(chunks)} 个文本块")  # 打印创建的块数
    return chunks  # 返回块列表


In [5]:
class SimpleVectorStore:
    """
    使用NumPy实现的简单向量存储。
    """
    def __init__(self):
        """
        初始化向量存储。
        """
        self.vectors = []  # 用于存储嵌入向量的列表
        self.texts = []  # 用于存储原始文本的列表
        self.metadata = []  # 用于存储每个文本元数据的列表

    def add_item(self, text, embedding, metadata=None):
        """
        向向量存储中添加一个项目。

        Args:
            text (str): 原始文本。
            embedding (List[float]): 嵌入向量。
            metadata (Dict, optional): 额外的元数据。
        """
        self.vectors.append(np.array(embedding))  # 将嵌入转换为numpy数组并添加到向量列表中
        self.texts.append(text)  # 将原始文本添加到文本列表中
        self.metadata.append(metadata or {})  # 添加元数据到元数据列表中，如果没有提供则使用空字典

    def add_items(self, texts, embeddings, metadata_list=None):
        """
        向向量存储中添加多个项目。

        Args:
            texts (List[str]): 文本内容列表
            embeddings (List[List[float]]): 嵌入向量列表
            metadata_list (List[Dict], optional): 元数据字典列表（可选）
        """
        # 如果未提供元数据列表，则为每个文本创建一个空字典
        if metadata_list is None:
            metadata_list = [{} for _ in range(len(texts))]

        # 将每个文本、嵌入和元数据添加到存储中
        for text, embedding, metadata in zip(texts, embeddings, metadata_list):
            self.add_item(text, embedding, metadata)


    def similarity_search(self, query_embedding, k=5):
        """
        查找与查询嵌入最相似的项目。

        Args:
            query_embedding (List[float]): 查询嵌入向量。
            k (int): 返回的结果数量。

        Returns:
            List[Dict]: 包含文本和元数据的前k个最相似项。
        """
        if not self.vectors:
            return []  # 如果没有存储向量，则返回空列表

        # 将查询嵌入转换为numpy数组
        query_vector = np.array(query_embedding)

        # 使用余弦相似度计算相似度
        similarities = []
        for i, vector in enumerate(self.vectors):
            # 计算查询向量与存储向量之间的余弦相似度
            similarity = np.dot(query_vector, vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
            similarities.append((i, similarity))  # 添加索引和相似度分数

        # 按相似度排序（降序）
        similarities.sort(key=lambda x: x[1], reverse=True)

        # 返回前k个结果
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],  # 添加对应的文本
                "metadata": self.metadata[idx],  # 添加对应的元数据
                "similarity": float(score)  # 添加相似度分数
            })

        return results  # 返回前k个最相似项的列表


In [6]:
def create_embeddings(texts):
    """
    为给定的文本创建嵌入向量。

    Args:
        texts (str 或 List[str]): 输入文本（可以是单个字符串或字符串列表）
        # model (str): 嵌入模型名称

    返回:
        List[List[float]]: 嵌入向量列表
    """
    # 处理字符串和列表类型的输入
    input_texts = texts if isinstance(texts, list) else [texts]

    # 如果需要，按批次处理（OpenAI API 有请求限制）
    batch_size = 100
    all_embeddings = []

    # 按批次迭代输入文本
    for i in range(0, len(input_texts), batch_size):
        batch = input_texts[i:i + batch_size]  # 获取当前批次的文本

        # 为当前批次创建嵌入向量
        response = client.embeddings.create(
            model=embedding_model,
            input=batch
        )

        # 从响应中提取嵌入向量
        batch_embeddings = [item.embedding for item in response.data]
        all_embeddings.extend(batch_embeddings)  # 将批次嵌入向量添加到总列表中

    # 如果输入是单个字符串，仅返回第一个嵌入向量
    if isinstance(texts, str):
        return all_embeddings[0]

    # 否则，返回所有嵌入向量
    return all_embeddings


## 命题生成

In [7]:
def generate_propositions(chunk):
    """
    从文本块中生成原子化、自包含的命题。

    Args:
        chunk (Dict): 包含内容和元数据的文本块

    Returns:
        List[str]: 生成的命题列表
    """
    # 系统提示，指示AI如何生成命题
    system_prompt = """请将以下文本分解为简单的自包含命题。确保每个命题符合以下标准：

    1. 表达单一事实：每个命题应陈述一个具体事实或主张
    2. 独立可理解：命题应自成体系，无需额外上下文即可理解
    3. 使用全称而非代词：避免使用代词或模糊指代，使用完整的实体名称
    4. 包含相关日期/限定条件：如适用应包含必要日期、时间和限定条件以保持准确性
    5. 保持单一主谓关系：聚焦单个主体及其对应动作或属性，避免连接词和多从句结构

    请仅输出命题列表，不要包含任何额外文本或解释，格式为命题列表。
    """
    # 用户提示，包含要转换为命题的文本块
    user_prompt = f"要转换为命题的文本:\n\n{chunk['text']}"

    # 从模型生成响应
    response = client.chat.completions.create(
        model=llm_model,  # 使用更强的模型以准确生成命题
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )

    # 从响应中提取命题
    raw_propositions = response.choices[0].message.content.strip().split('\n')

    # 清理命题（移除编号、项目符号等）
    clean_propositions = []
    for prop in raw_propositions:
        # 移除编号（如1., 2.等）和项目符号
        cleaned = re.sub(r'^\s*(\d+\.|\-|\*)\s*', '', prop).strip()
        if cleaned and len(cleaned) > 10:  # 简单过滤空或过短的命题
            clean_propositions.append(cleaned)

    return clean_propositions


## 命题质量核查

In [8]:
def evaluate_proposition(proposition, original_text):
    """
    根据准确性、清晰性、完整性以及简洁性评估命题的质量。

    Args:
        proposition (str): 要评估的命题
        original_text (str): 用于比较的原文

    Returns:
        Dict: 每个评估维度的分数
    """
    # 系统提示，指示AI如何评估命题
    system_prompt = """你是一位评估从文本中提取命题质量的专家。请根据以下标准对给定命题进行评分（1-10分）：

    - 准确性（Accuracy）：命题反映原文信息的准确程度
    - 清晰性（Clarity）：不依赖额外上下文的情况下，命题是否易于理解
    - 完整性（Completeness）：命题是否包含必要的细节（如日期、限定词等）
    - 简洁性（Conciseness）：命题是否在保留关键信息前提下，表述精简程度

    响应必须为有效的JSON格式，并包含每个标准的数值评分：
    {"accuracy": X, "clarity": X, "completeness": X, "conciseness": X}
    """

    # 用户提示，包含命题和原文
    user_prompt = f"""命题: {proposition}

    原文: {original_text}

    请以JSON格式提供你的评分。"""

    # 从模型生成响应
    response = client.chat.completions.create(
        model=llm_model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        response_format={"type": "json_object"},
        temperature=0
    )

    # 解析JSON响应
    try:
        scores = json.loads(response.choices[0].message.content.strip())
        return scores
    except json.JSONDecodeError:
        # 如果JSON解析失败，使用默认值作为回退
        return {
            "accuracy": 5,
            "clarity": 5,
            "completeness": 5,
            "conciseness": 5
        }


## 完整的命题处理流程

In [9]:
def process_document_into_propositions(pdf_path, chunk_size=800, chunk_overlap=100, quality_thresholds=None):
    """
    将文档处理为经过质量检查的命题。

    Args:
        pdf_path (str): PDF文件的路径
        chunk_size (int): 每个分块的字符大小
        chunk_overlap (int): 分块之间的重叠字符数
        quality_thresholds (Dict): 命题质量的阈值分数

    Returns:
        Tuple[List[Dict], List[Dict]]: 原始分块和命题分块
    """
    # 如果未提供，则设置默认的质量阈值
    if quality_thresholds is None:
        quality_thresholds = {
            "accuracy": 7,  # 准确性阈值
            "clarity": 7,   # 清晰性阈值
            "completeness": 7,  # 完整性阈值
            "conciseness": 7  # 简洁性阈值
        }

    # 从PDF文件中提取文本
    text = extract_text_from_pdf(pdf_path)

    # 从提取的文本创建分块
    chunks = chunk_text(text, chunk_size, chunk_overlap)

    # 初始化一个列表以存储所有命题
    all_propositions = []

    print("从分块生成命题...")
    for i, chunk in enumerate(chunks):
        print(f"处理分块 {i+1}/{len(chunks)}...")

        # 为当前分块生成命题
        chunk_propositions = generate_propositions(chunk)
        print(f"生成了 {len(chunk_propositions)} 个命题")

        # 处理每个生成的命题
        for prop in chunk_propositions:
            proposition_data = {
                "text": prop,  # 命题文本
                "source_chunk_id": chunk["chunk_id"],  # 来源分块ID
                "source_text": chunk["text"]  # 来源分块文本
            }
            all_propositions.append(proposition_data)

    # 评估生成的命题质量
    print("\n评估命题质量...")
    quality_propositions = []

    for i, prop in enumerate(all_propositions):
        if i % 10 == 0:  # 每10个命题进行一次状态更新
            print(f"评估命题 {i+1}/{len(all_propositions)}...")

        # 评估当前命题的质量
        scores = evaluate_proposition(prop["text"], prop["source_text"])
        prop["quality_scores"] = scores

        # 检查命题是否通过质量阈值
        passes_quality = True
        for metric, threshold in quality_thresholds.items():
            if scores.get(metric, 0) < threshold:
                passes_quality = False
                break

        if passes_quality:
            quality_propositions.append(prop)
        else:
            print(f"命题未通过质量检查: {prop['text'][:50]}...")

    print(f"\n在质量过滤后保留了 {len(quality_propositions)}/{len(all_propositions)} 个命题")

    return chunks, quality_propositions


## 为两种方法构建向量存储

In [10]:
def build_vector_stores(chunks, propositions):
    """
    为基于分块和基于命题的两种方法构建向量存储。

    Args:
        chunks (List[Dict]): 原始文档分块
        propositions (List[Dict]): 经过质量过滤的命题

    Returns:
        Tuple[SimpleVectorStore, SimpleVectorStore]: 分块和命题的向量存储
    """
    # 创建用于分块的向量存储
    chunk_store = SimpleVectorStore()

    # 提取分块文本并创建嵌入
    chunk_texts = [chunk["text"] for chunk in chunks]
    print(f"为 {len(chunk_texts)} 个分块创建嵌入...")
    chunk_embeddings = create_embeddings(chunk_texts)

    # 将分块添加到向量存储中，并附带元数据
    chunk_metadata = [{"chunk_id": chunk["chunk_id"], "type": "chunk"} for chunk in chunks]
    chunk_store.add_items(chunk_texts, chunk_embeddings, chunk_metadata)

    # 创建用于命题的向量存储
    prop_store = SimpleVectorStore()

    # 提取命题文本并创建嵌入
    prop_texts = [prop["text"] for prop in propositions]
    print(f"为 {len(prop_texts)} 个命题创建嵌入...")
    prop_embeddings = create_embeddings(prop_texts)

    # 将命题添加到向量存储中，并附带元数据
    prop_metadata = [
        {
            "type": "proposition",
            "source_chunk_id": prop["source_chunk_id"],
            "quality_scores": prop["quality_scores"]
        }
        for prop in propositions
    ]
    prop_store.add_items(prop_texts, prop_embeddings, prop_metadata)

    return chunk_store, prop_store


## 查询和检索函数

In [11]:
def retrieve_from_store(query, vector_store, k=5):
    """
    根据查询从向量存储中检索相关项。

    Args:
        query (str): 用户查询
        vector_store (SimpleVectorStore): 要搜索的向量存储
        k (int): 要检索的结果数量

    Returns:
        List[Dict]: 检索到的项目及其分数和元数据
    """
    # 创建查询嵌入
    query_embedding = create_embeddings(query)

    # 在向量存储中搜索与查询最相似的前 k 个项目
    results = vector_store.similarity_search(query_embedding, k=k)

    return results


In [12]:
def compare_retrieval_approaches(query, chunk_store, prop_store, k=5):
    """
    比较基于块和基于命题的检索方法对查询的结果。

    Args:
        query (str): 用户查询
        chunk_store (SimpleVectorStore): 基于块的向量存储
        prop_store (SimpleVectorStore): 基于命题的向量存储
        k (int): 从每个存储中检索结果的数量

    Returns:
        Dict: 比较结果
    """
    print(f"\n=== 查询: {query} ===")

    # 从基于命题的向量存储中检索结果
    print("\n使用基于命题的方法检索...")
    prop_results = retrieve_from_store(query, prop_store, k)

    # 从基于块的向量存储中检索结果
    print("使用基于块的方法检索...")
    chunk_results = retrieve_from_store(query, chunk_store, k)

    # 显示基于命题的结果
    print("\n=== 基于命题的结果 ===")
    for i, result in enumerate(prop_results):
        print(f"{i+1}) {result['text']} (分数: {result['similarity']:.4f})")

    # 显示基于块的结果
    print("\n=== 基于块的结果 ===")
    for i, result in enumerate(chunk_results):
        # 截断文本以保持输出可管理
        truncated_text = result['text'][:150] + "..." if len(result['text']) > 150 else result['text']
        print(f"{i+1}) {truncated_text} (分数: {result['similarity']:.4f})")

    # 返回比较结果
    return {
        "query": query,
        "proposition_results": prop_results,
        "chunk_results": chunk_results
    }


## 回答生成与评估

In [13]:
def generate_response(query, results, result_type="proposition"):
    """
    基于检索到的结果生成响应。

    Args:
        query (str): 用户查询
        results (List[Dict]): 检索到的项目
        result_type (str): 结果类型 ('proposition' 或 'chunk')

    Returns:
        str: 生成的响应
    """
    # 将检索到的文本合并为一个单一的上下文字符串
    context = "\n\n".join([result["text"] for result in results])

    # 系统提示，指示AI如何根据检索到的信息生成响应
    system_prompt = f"""您是一个基于检索信息回答问题的AI助手。
您的答案应基于从知识库中检索到的以下 {result_type}。
如果检索到的信息无法回答问题，请明确指出这一限制。"""

    # 用户提示，包含查询和检索到的上下文
    user_prompt = f"""查询: {query}

    检索到的 {result_type}:
    {context}

    请根据检索到的信息回答查询。"""

    # 使用OpenAI客户端生成响应
    response = client.chat.completions.create(
        model=llm_model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.2
    )

    # 返回生成的响应文本
    return response.choices[0].message.content


In [14]:
def evaluate_responses(query, prop_response, chunk_response, reference_answer=None):
    """
    评估并比较两种方法生成的响应。

    Args:
        query (str): 用户查询
        prop_response (str): 命题检索方法生成的响应
        chunk_response (str): 块检索方法生成的响应
        reference_answer (str, 可选): 用于比较的参考答案

    Returns:
        str: 评估分析
    """
    # 系统提示，指导AI如何评估两种响应
    system_prompt = """您是信息检索系统的专业评估员。请比较对同一查询的两种响应（基于命题检索和基于段落块检索的版本）。

    根据以下标准进行评估：
    1. 准确性：哪个回答提供的事实信息更正确？
    2. 相关性：哪个回答更贴合具体查询需求？
    3. 简洁性：哪个回答在保持完整性的前提下更加简洁？
    4. 清晰度：哪个回答更容易理解？

    具体说明每种方法的优点和缺点。"""

    # 用户提示，包含查询和要比较的两种响应
    user_prompt = f"""查询: {query}

    命题检索方法生成的回答:
    {prop_response}

    块检索方法生成的回答:
    {chunk_response}"""

    # 如果提供了参考答案，则将其包含在用户提示中以进行事实核查
    if reference_answer:
        user_prompt += f"""

    参考答案（用于事实核查）:
    {reference_answer}"""

    # 添加最终指示到用户提示
    user_prompt += """
    请提供详细的回答对比分析，明确指出哪种方法表现更好及其原因。"""

    # 使用OpenAI客户端生成评估分析
    response = client.chat.completions.create(
        model=llm_model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )

    # 返回生成的评估分析
    return response.choices[0].message.content


## 完整的端到端的评估流程

In [15]:
def run_proposition_chunking_evaluation(pdf_path, test_queries, reference_answers=None):
    """
    运行命题分块与标准分块的完整评估。

    Args:
        pdf_path (str): PDF 文件路径
        test_queries (List[str]): 测试查询列表
        reference_answers (List[str], 可选): 查询的参考答案

    Returns:
        Dict: 评估结果
    """
    print("=== 开始命题分块评估 ===\n")

    # 将文档处理为命题和块
    chunks, propositions = process_document_into_propositions(pdf_path)

    # 为块和命题构建向量存储
    chunk_store, prop_store = build_vector_stores(chunks, propositions)

    # 初始化一个列表以存储每个查询的结果
    results = []

    # 对每个查询运行测试
    for i, query in enumerate(test_queries):
        print(f"\n\n=== 测试查询 {i+1}/{len(test_queries)} ===")
        print(f"查询: {query}")

        # 从基于块和基于命题的方法中获取检索结果
        retrieval_results = compare_retrieval_approaches(query, chunk_store, prop_store)

        # 根据检索到的命题结果生成响应
        print("\n从命题结果生成响应...")
        prop_response = generate_response(
            query,
            retrieval_results["proposition_results"],
            "proposition"
        )

        # 根据检索到的块结果生成响应
        print("从块结果生成响应...")
        chunk_response = generate_response(
            query,
            retrieval_results["chunk_results"],
            "chunk"
        )

        # 如果可用，获取参考答案
        reference = None
        if reference_answers and i < len(reference_answers):
            reference = reference_answers[i]

        # 评估生成的响应
        print("\n评估响应...")
        evaluation = evaluate_responses(query, prop_response, chunk_response, reference)

        # 编译当前查询的结果
        query_result = {
            "query": query,
            "proposition_results": retrieval_results["proposition_results"],
            "chunk_results": retrieval_results["chunk_results"],
            "proposition_response": prop_response,
            "chunk_response": chunk_response,
            "reference_answer": reference,
            "evaluation": evaluation
        }

        # 将结果附加到整体结果列表中
        results.append(query_result)

        # 打印当前查询的响应和评估
        print("\n=== 命题方法响应 ===")
        print(prop_response)

        print("\n=== 块方法响应 ===")
        print(chunk_response)

        print("\n=== 评估 ===")
        print(evaluation)

    # 生成评估的整体分析
    print("\n\n=== 生成整体分析 ===")
    overall_analysis = generate_overall_analysis(results)
    print("\n" + overall_analysis)

    # 返回评估结果、整体分析以及命题和块的数量
    return {
        "results": results,
        "overall_analysis": overall_analysis,
        "proposition_count": len(propositions),
        "chunk_count": len(chunks)
    }


In [16]:
def generate_overall_analysis(results):
    """
    生成命题方法与块方法的整体分析。

    Args:
        results (List[Dict]): 每个测试查询的结果

    Returns:
        str: 整体分析
    """
    # 系统提示，指导AI如何生成整体分析
    system_prompt = """
    您是信息检索系统的评估专家。请基于多个测试查询，提供比较基于命题检索与基于块检索在RAG（检索增强生成）系统中的整体分析。

    分析重点应包含以下几点：
    1. 命题检索方法在哪种情况下表现更好
    2. 块检索方法在哪种情况下表现更好
    3. 两种方法的整体优缺点对比
    4. 不同场景下的使用建议"""

    # 创建每个查询的评估摘要
    evaluations_summary = ""
    for i, result in enumerate(results):
        evaluations_summary += f"查询 {i+1}: {result['query']}\n"
        evaluations_summary += f"评估摘要: {result['evaluation'][:200]}...\n\n"

    # 用户提示，包含评估摘要
    user_prompt = ""
    user_prompt = f"""根据以下针对{len(results)}个查询的命题检索与段落块检索对比评估，请提供两种方法的整体分析：

    {evaluations_summary}

    请提供一份全面的分析，说明命题检索方法与块检索方法在RAG系统中的相对优势与不足。"""

    # 使用OpenAI客户端生成整体分析
    response = client.chat.completions.create(
        model=llm_model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )

    # 返回生成的分析文本
    return response.choices[0].message.content


## 命题分块的评估

In [17]:
# 定义测试查询，涵盖 AI 的不同方面，用于评估命题分块的效果
test_queries = [
    "AI 开发中的主要伦理问题是什么？",
    # "可解释性 AI 如何提高用户对 AI 系统的信任？",
    # "开发公平 AI 系统的关键挑战有哪些？",
    # "人类监督在 AI 安全中扮演什么角色？"
]

# 为更深入的评估和结果比较提供参考答案
# 这些参考答案提供了衡量生成响应质量的基准
reference_answers = [
    "AI 开发中的主要伦理问题包括偏见与公平性、隐私、透明度、责任、安全以及可能被滥用或产生有害应用的风险。",
    # "可解释性 AI 通过使 AI 决策过程对用户透明且易于理解来增强信任，帮助用户验证公平性、识别潜在偏差，并更好地理解 AI 的局限性。",
    # "开发公平 AI 系统的关键挑战包括解决数据偏差、确保训练数据的多样性表示、创建透明算法、定义不同上下文中的公平性，并平衡竞争性的公平标准。",
    # "人类监督在 AI 安全中起着关键作用，通过监控系统行为、验证输出、必要时进行干预、设定伦理边界，并确保 AI 系统在整个运行过程中与人类价值观和意图保持一致。"
]

# 运行评估
evaluation_results = run_proposition_chunking_evaluation(
    pdf_path=pdf_path,  # 输入的 PDF 文件路径
    test_queries=test_queries,  # 测试查询列表
    reference_answers=reference_answers  # 参考答案列表
)

# 打印整体分析结果
print("\n\n=== 整体分析 ===")
print(evaluation_results["overall_analysis"])  # 输出评估的整体分析部分


=== 开始命题分块评估 ===

创建了 15 个文本块
从分块生成命题...
处理分块 1/15...
生成了 40 个命题
处理分块 2/15...
生成了 57 个命题
处理分块 3/15...
生成了 77 个命题
处理分块 4/15...
生成了 49 个命题
处理分块 5/15...
生成了 57 个命题
处理分块 6/15...
生成了 59 个命题
处理分块 7/15...
生成了 55 个命题
处理分块 8/15...
生成了 60 个命题
处理分块 9/15...
生成了 56 个命题
处理分块 10/15...
生成了 55 个命题
处理分块 11/15...
生成了 55 个命题
处理分块 12/15...
生成了 58 个命题
处理分块 13/15...
生成了 63 个命题
处理分块 14/15...
生成了 53 个命题
处理分块 15/15...
生成了 20 个命题

评估命题质量...
评估命题 1/814...
命题未通过质量检查: 人工智能是指数字计算机或计算机控制的机器人执行通常与智能生物相关的任务的能力。...
命题未通过质量检查: 人工智能这一术语通常用于开发具有人类特有的智力过程的系统。...
命题未通过质量检查: 人类特有的智力过程包括推理、发现意义、概括或从过往经验中学习的能力。...
命题未通过质量检查: 在过去的几十年中，计算能力和数据可用性的进步显著加速了人工智能的开发和部署。...
命题未通过质量检查: 人工智能的概念已存在数个世纪。...
命题未通过质量检查: 人工智能经常出现在神话和小说中。...
命题未通过质量检查: 人工智能研究的正式领域始于20世纪中叶。...
命题未通过质量检查: 1956年的达特茅斯研讨会被广泛认为是人工智能的发源地。...
命题未通过质量检查: 早期的人工智能研究侧重于问题解决和符号方法。...
命题未通过质量检查: 20世纪80年代专家系统兴起。...
评估命题 11/814...
命题未通过质量检查: 20世纪90年代和21世纪初，机器学习和神经网络取得了进步。...
命题未通过质量检查: 深度学习的最新突破彻底改变了人工智能领域。...
命题未通过质量检查: 现代人工智能系统在日常生活中的普及程度日益提高。...
命题未通过质量检查: Siri和Alexa是虚拟助手的

In [18]:
print(json.dumps(evaluation_results, indent=4, ensure_ascii=False))

{
    "results": [
        {
            "query": "AI 开发中的主要伦理问题是什么？",
            "proposition_results": [],
            "chunk_results": [
                {
                    "text": "⼈⼯智能的开发和部署。这些框架旨在解\n决伦理问题，促进创新，并确保负责任的⼈⼯智能实践。\n⼈⼯智能监管\n⼈⼯智能监管是⼀个复杂且不断发展的领域。各国政府正在考虑制定法规，以解决偏⻅、透明度、\n隐私和安全等问题。平衡创新与伦理考量是⼀项关键挑战。\n⼈⼯智能研发资⾦\n政府在资助⼈⼯智能研发⽅⾯发挥着⾄关重要的作⽤。公共资⾦⽀持基础研究、应⽤研究以及⼈⼯\n智能基础设施的建设。政府投资推动创新，促进合作。\n国际合作\n国际合作对于应对⼈⼯智能带来的全球挑战和机遇⾄关重要。这包括共享知识、制定标准以及跨境\n推⼴负责任的⼈⼯智能实践。\n公众参与和教育\n让公众参与⼈⼯智能讨论，对于建⽴信任并确保⼈⼯智能发展符合社会价值观⾄关重要。教育和宣\n传活动可以让公众了解⼈⼯智能、其影响及其潜⼒。\n第 19 章：⼈⼯智能与伦理\n道德⼈⼯智能原则\n符合伦理道德的⼈⼯智能原则指导着⼈⼯智能系统的开发和部署，以确保其公平、透明、负责且有\n益于社会。关键原则包括尊重⼈权、隐私、不歧视和仁慈。\n解决⼈⼯智能中的偏⻅\n⼈⼯智能系统可能会继承并放⼤其训练数据中存在的偏⻅，从⽽导致不公平或歧视性的结果。解决\n偏⻅需要谨慎的数据收集、算法设计以及持续的监测和评估。\n透明度和可解释性\n透明度和可解释性对于建⽴对⼈⼯智能系统的信任⾄关重要。可解释⼈⼯智能 (XAI) 技术旨在使⼈\n⼯智能决策更易于理解，使⽤⼾能够评估其公平性和准确性。\n隐私和数据保护\n⼈⼯智能系统通常依赖⼤量数据，这引发了⼈们对隐私和数据保护的担忧。确保负责任的数据处\n理、实施隐私保护技术以及遵守数据保护法规⾄关重要。\n问责与责任\n建⽴⼈⼯智能系统的问责制和责任制，对于应对潜在危害和确保道德⾏为⾄关重要。这包括明确⼈\n⼯智能系统开发者、部署者和⽤⼾的⻆⾊和职责。\n第 20 章：建⽴对⼈⼯智能的信任\n透明度和可解释性\n

In [20]:
# !注意：此方法特别消耗token，适合小文件

total_proposition = 40 + 57 + 77 + 49 + 57 + 59 + 55 + 60 + 56 + 55 + 55 + 58 + 63 + 53 + 20
print(total_proposition)

814
